In [1]:
import torch
from torch import nn
from torch.nn import functional as F
import torchvision
from torchvision import transforms
from torch import optim


In [2]:
# 训练函数
def train_epoch(net, data_loader, device):
    net.train() # 指定当前为训练模式
    train_batch_num = len(data_loader) # 记录共有多少个batch
    total_loss = 0 # 记录loss
    correct = 0 # 记录共有多少个样本被正确分类
    sample_num = 0 # 记录样本总数
    
    # 遍历每个batch进行训练
    for batch_idx, (data, target) in enumerate(data_loader):
        data = data.to(device).float() # 将图片放入指定的device中
        target = target.to(device).long() # 将图片标签放入到指定的device中
        optimizer.zero_grad() # 将当前梯度清零
        output = net(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        prediction = torch.argmax(output, 1) # 找出每个样本值最大的idx, 即岱庙预测此图片属于哪个类别
        correct += (prediction == target).sum().item()
        sample_num += len(prediction)
    
    loss = total_loss / train_batch_num
    acc = correct / sample_num
    return loss, acc

In [3]:
# 测试函数
def test_epoch(net, data_loader, device):
    net.eval() # 指定当前模式为测试模式
    test_batch_num = len(data_loader)
    total_loss = 0
    correct = 0
    sample_num = 0
    
    # 指定不进行梯度变化
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(data_loader):
            data = data.to(device).float()
            target = target.to(device).long()
            output = net(data)
            loss = criterion(output, target)
            total_loss += loss.item()
            prediction = torch.argmax(output, 1)
            correct += (prediction == target).sum().item()
            sample_num += len(prediction)
            
        loss = total_loss / test_batch_num
        acc = correct / sample_num
        return loss, acc

In [4]:
# 空洞卷积实现
class DilatedConvModule(nn.Module):
    def __init__(self):
        super(DilatedConvModule, self).__init__()
        # 定义一个空洞率为1，2，5的三层空洞卷积
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=0, dilation=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=0, dilation=2),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=0, dilation=5),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True)
        )
        # 输出层，将通道数变为分类数量
        self.fc = nn.Linear(128, num_classes)
        
    def forward(self, X):
        # 图片先经过三层空洞卷积
        out = self.conv(X)
        # 使用平均池化层将图片的大小变为1x1 out尺寸[512 128 16 16]
        out = F.avg_pool2d(out, 16)
        # 将张量out从shape batchx128x1x1 变为 batchx128
        out = out.squeeze()
        # 输入到全连接层将输入的维度变为10
        out = self.fc(out)
        return out

In [5]:
# 读取数据
data_dir = './data'

# 定义一个transform操作，用户将torch中的数据转换为可以输入到我们模式的形式
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)) 
    ]
) # 将数据归一化

#获取cifar-10数据集并进行transform
cifar_train = torchvision.datasets.CIFAR10(root=data_dir, train=True, download=True,transform=transform)
cifar_test = torchvision.datasets.CIFAR10(root=data_dir, train=False, download=True,transform=transform)

# cifar-10数据集对应10个类别
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

num_classes = 10
epochs = 1
lr = 0.001
batch_size = 512
device = torch.device('cpu') # 如使用0号GPU则填写 ‘cuda:1’

#生成dataloader
cifar_trainloader = torch.utils.data.DataLoader(cifar_train, batch_size=batch_size, shuffle=True, num_workers = 0)
cifar_testloader = torch.utils.data.DataLoader(cifar_test, batch_size=512, shuffle=True, num_workers=0)

print(len(cifar_trainloader))
print(len(cifar_testloader))

# 初始化模型
net = DilatedConvModule().to(device)

# 使用多元交叉熵损失
criterion = nn.CrossEntropyLoss()

# 使用Adam优化器
optimizer = optim.Adam(net.parameters(), lr=lr)

Files already downloaded and verified
Files already downloaded and verified
98
20


In [6]:
# 训练模型
train_loss_list = []
train_acc_list = []
test_loss_list = []
test_acc_list = []

for epoch in range(epochs):
    # 在训练集上训练
    train_loss, train_acc = train_epoch(net, data_loader = cifar_trainloader, device = device)
    # 在测试集上测试
    test_loss, test_acc = test_epoch(net, data_loader=cifar_testloader, device = device)
    # 保存各指标
    train_loss_list.append(train_loss)
    train_acc_list.append(train_acc)
    test_loss_list.append(test_loss)
    test_acc_list.append(test_acc)
    print(f'epoch: {epoch}\t train_loss: {train_loss:.4f} \t'
          f'train_acc:{train_acc}\t'
          f'test_loss:{test_loss:.4f} \t test_acc:{test_acc}')

RuntimeError: [enforce fail at CPUAllocator.cpp:71] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 88604672 bytes. Error code 12 (Cannot allocate memory)

In [ ]:
#画图
x = np.linspace(0, len(train_loss_list), len(train_loss_list))
plt.plot(x, train_loss_list, label = 'train_loss', linewidth=1.5)
plt.plot(x, test_loss_list, label='test_loss', linewidth = 1.5)
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend()
plt.show()

y = np.linspace(0, len(train_acc_list), len(train_acc_list))
plt.plot(y, train_acc_list, label = 'train_acc', linewidth=1.5)
plt.plot(y, test_acc_list, label='test_acc', linewidth = 1.5)
plt.xlabel('epoch')
plt.ylabel('acc')
plt.legend()
plt.show()